In [5]:
import pandas as pd
import numpy as np
import torch
import sklearn
import matplotlib
import torchinfo, torchmetrics

# Check PyTorch access (should print out a tensor)
print(torch.randn(3, 3))

# Check for GPU (should return True)
print(torch.cuda.is_available())

tensor([[-1.0720, -1.1588, -0.6223],
        [ 1.3196, -0.1108, -1.1896],
        [-0.9192,  1.4755,  0.6995]])
False


In [13]:
print(torch.__version__)

2.7.0


In [6]:
# Crea un tensore casuale
x = torch.randn(3, 3)
print(f"Random tensor:\n{x}")

# Verifica disponibilità acceleratori
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS available: {torch.backends.mps.is_available() if hasattr(torch.backends, 'mps') else 'MPS module not found'}")

# Prova a utilizzare MPS se disponibile
if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
    x = x.to(device)
    print(f"Tensor moved to MPS device: {x.device}")

Random tensor:
tensor([[-1.1899,  0.1600, -0.3878],
        [-0.5169, -0.7166, -0.3784],
        [-1.3063,  0.3814,  1.7909]])
CUDA available: False
MPS available: True
Tensor moved to MPS device: mps:0


## 🚨Durante il corso sostituire CUDA CON MPS🚨

`device = torch.device("cuda" if torch.cuda.is_available() else "cpu")`

diventerà

`device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")`

### Funzione per rendere universale?

In [7]:
def get_device():
    if torch.backends.mps.is_available():
        return torch.device("mps")
    elif torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

# Poi usa
device = get_device()

In [8]:
print(device)

mps


## Introduction to Tensors
### Creating tensors

I tensori PyTotch sono creati usando `torch.Tensor()` [documentazione](https://pytorch.org/docs/stable/tensors.html)

In [9]:
# scalar (non ha dimensione)

scalar = torch.tensor(7)
scalar

tensor(7)

In [12]:
scalar.ndim

0

In [14]:
# Get tensor back as Python int
scalar.item()

7

### Vector 
Array **unidimensionale** di numeri. Rappresenta una grandezza con magnitudine e direzione.
**due caratteristiche di un vettore**
* **Magnitudine (o Modulo)**: È la lunghezza o grandezza del vettore. Rappresenta quanto è "lungo" il vettore nello spazio. Per un vettore v in uno spazio n-dimensionale, la magnitudine si calcola come la radice quadrata della somma dei quadrati delle sue componenti:
$||v|| = √(v₁² + v₂² + ... + vₙ²)$
* **Direzione**: È l'orientamento del vettore nello spazio. Indica verso dove punta il vettore ed è determinata dagli angoli che il vettore forma con gli assi di riferimento. Un vettore unitario (con magnitudine 1) nella stessa direzione del vettore originale può essere ottenuto dividendo il vettore per la sua magnitudine:
$û = v / ||v||$

Queste due caratteristiche insieme definiscono completamente un vettore: quanto è lungo (magnitudine) e verso dove punta (direzione). In PyTorch e nel machine learning, queste proprietà sono fondamentali per molte operazioni, come la normalizzazione dei vettori, il calcolo di distanze e similitudini, e l'ottimizzazione dei gradienti.

In [15]:
# vector
vector = torch.tensor([7,7])
vector

tensor([7, 7])

In [16]:
vector.ndim

1

In [17]:
vector.shape

torch.Size([2])

### Matrix
Array **bidimensionale** di numeri organizzati in righe e colonne. Trasforma vettori in altri vettori.
Dimensioni: Una matrice è caratterizzata dal numero di righe e colonne (m×n), che definiscono la sua forma.

In [18]:
MATRIX = torch.tensor([[7, 8], 
                       [9, 10]])
MATRIX

tensor([[ 7,  8],
        [ 9, 10]])

In [19]:
MATRIX.ndim

2

In [20]:
MATRIX[1]

tensor([ 9, 10])

In [22]:
MATRIX.shape

torch.Size([2, 2])

## TENSOR
Generalizzazione **multidimensionale** di vettori e matrici. Un array n-dimensionale di numeri. Un vettore è un tensore di rango 1, una matrice è un tensore di rango 2, mentre un tensore può avere rango 3 o superiore. 

Esempio di tensore 3D: `[[[1, 2], [3, 4]], [[5, 6], [7, 8]]]`.

Nel deep learning, i tensori sono l'unità fondamentale di dati: immagini (tensori 4D), sequenze di testo (tensori 3D), pesi delle reti (tensori di vari ranghi).

In [23]:
TENSOR = torch.tensor([[[1,2,3],
                       [3,6,9],
                       [2,3,4]]])
TENSOR

tensor([[[1, 2, 3],
         [3, 6, 9],
         [2, 3, 4]]])

In [25]:
TENSOR.ndim

3

In [26]:
TENSOR.shape

torch.Size([1, 3, 3])

In [27]:
TENSOR[0]

tensor([[1, 2, 3],
        [3, 6, 9],
        [2, 3, 4]])

In [31]:
TENSOR[0][1]

tensor([3, 6, 9])

In [32]:
TENSOR[0][1][2]

tensor(9)

| **Name** | **What is it?** | **Number of dimensions** | **Lower or upper (usually/example)** |
|----------|------------|--------------------------|-------------------------------------|
| **scalar** | a single number | 0 | Lower (a) | 
| **vector** | a number with direction (e.g. wind speed with direction) but can also have many other numbers | 1 | Lower (y) |
| **matrix** | a 2-dimensional array of numbers | 2 | Upper (Q) |
| **tensor** | an n-dimensional array of numbers | can be any number, a 0-dimension tensor is a scalar, a 1-dimension tensor is a vector | Upper (X) |

## Random Tensors

I tensori randomici sono importanti perchè il modo in cui le reti neurali imparano, è iniziare con un tensore pieno di numeri randomici e poi aggiustano questi numeri per una rappresentazione migliore dei dati.

`Start with random numbers -> look at data -> update random numbers -> look at data -> update random numbers...`

In [34]:
# Create a random tensor of shape(size) (3,4)

random_tensor = torch.rand(3,4)
random_tensor

tensor([[0.1070, 0.0685, 0.2837, 0.4913],
        [0.0442, 0.1371, 0.0126, 0.7032],
        [0.7393, 0.0785, 0.3405, 0.2403]])

In [35]:
random_tensor.ndim

2

In [36]:
random_tensor.shape

torch.Size([3, 4])

In [42]:
# create a random tensor with a similar shape to an image
random_image_size_tensor = torch.rand(size=(3, 224,224)) # colour channels (R,G,B), height, width, 
random_image_size_tensor.shape, random_image_size_tensor.ndim

(torch.Size([3, 224, 224]), 3)

![tensore](image/00-tensor-shape-example-of-image.png)

## Zeros and Ones

In [45]:
# Creating Tensor of all zeros (esempio di mask)
zeros = torch.zeros(size=(3,4))
zeros

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [48]:
# Creating Tensor of all ones
ones = torch.ones(size=(3,4))
ones

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [49]:
ones.dtype

torch.float32

## Creating a range of tensors and tensors-like

In [61]:
# torch.arange(start, end, step)
one_to_ten = torch.arange(start=0,end=11, step=1)
one_to_ten

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [60]:
# Creating a tensor like
ten_zeroes = torch.zeros_like(input=one_to_ten)
ten_zeroes

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Il comando `torch.zeros_like(input=one_to_ten)` crea un nuovo tensore con:

* La stessa forma (dimensioni) del tensore one_to_ten
* Lo stesso tipo di dati (dtype) del tensore one_to_ten
* Lo stesso dispositivo (CPU o GPU) del tensore one_to_ten

Ma con tutti gli elementi inizializzati a zero.
Per esempio:

- Se `one_to_ten` è un vettore di lunghezza 10, ten_zeroes sarà un vettore di lunghezza 10 contenente solo zeri
- Se `one_to_ten` è una matrice 3×4, ten_zeroes sarà una matrice 3×4 contenente solo zeri

**Questo è molto utile quando hai bisogno di creare un tensore con le stesse caratteristiche di un tensore esistente, ma con valori diversi**. Altri metodi simili includono:

- `torch.ones_like()`: come zeros_like ma riempie con 1
- `torch.randn_like()`: stessa forma ma con valori casuali da una distribuzione normale
- `torch.rand_like()`: stessa forma ma con valori casuali uniformi tra 0 e 1
- `torch.empty_like()`: stessa forma ma senza inizializzare i valori (per performance)

Se invece volessi creare un tensore con la stessa forma ma con un dtype o dispositivo diverso, potresti specificarlo:

`pythonten_zeroes = torch.zeros_like(one_to_ten, dtype=torch.float32, device="mps")`

## Tensor Datatypes
- Il `dtype` è per la **precision in computing**
    - **Precision** is the amount of detail used to describe a number.
      The higher the precision value (8, 16, 32), the more detail and hence data
      used to express a number. This matters in deep learning and numerical computing because you're making so many operations, the more detail you have to calculate on, the more compute you have to use. So lower precision datatypes are generally faster to compute on but sacrifice some performance on evaluation metrics like accuracy (faster to compute but less accurate).
- `device` what device is the tensor stored on? (usually GPU or CPU)
- `require_grad` se vogliamo o meno calcolare i gradienti

In [64]:
# Float 32 tensor (è il default)
float_32_tensor = torch.tensor([3.0, 6.0, 9.0],
                              dtype=None, # What datatype is the tensor (e.g. float32 or float16)
                              device=None, # defaults to None, which uses the default tensor type
                              requires_grad=False) # if True, operations performed on the tensor are recorded
float_32_tensor

tensor([3., 6., 9.])

In [63]:
float_32_tensor.dtype

torch.float32

In [65]:
float_16_tensor = float_32_tensor.type(torch.float16)
float_16_tensor

tensor([3., 6., 9.], dtype=torch.float16)

## Getting Tensor Attributes
- check tensor datatype `tensor.dtype`
- check tensor shape `tensor.shape`
- check tensor device(GPU|TPU) `tensor.device`

In [66]:
# Create a tensor
some_tensor = torch.rand(3,4)
some_tensor

tensor([[0.7819, 0.0979, 0.0121, 0.7729],
        [0.7100, 0.4248, 0.3079, 0.2666],
        [0.6290, 0.9386, 0.5207, 0.8704]])

In [67]:
# Find out details about some tensor
print(some_tensor)
print(f"Datatype of tensor: {some_tensor.dtype}")
print(f"Shape of tensor: {some_tensor.shape}")
print(f"Device tensor is on: {some_tensor.device}")

tensor([[0.7819, 0.0979, 0.0121, 0.7729],
        [0.7100, 0.4248, 0.3079, 0.2666],
        [0.6290, 0.9386, 0.5207, 0.8704]])
Datatype of tensor: torch.float32
Shape of tensor: torch.Size([3, 4])
Device tensor is on: cpu
